In [1]:
import os
import sys
from selenium import webdriver
import time
import re

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Leverage Selenium to Login and Extract Charges

In [2]:
#####################################################################################################################
# open some required information from saved text file
#####################################################################################################################

with open('info.txt', 'r') as f:
    chars_to_remove = f.readline()
    password = f.readline()
chars_to_remove = chars_to_remove[0]

month_to_year_dict = {
    'Oct' : '2022',
    'Nov' : '2022',
    'Dec' : '2022',
    'Jan' : '2023'
}

In [3]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://member.neofinancial.com/login')

In [4]:
# click into email section
email_field = driver.find_element("xpath", '/html/body/div[1]/main/div[2]/div[1]/div[2]/div/div/form/div[1]/div[1]/label')
time.sleep(1.1)
email_field.click()

time.sleep(1.1)

# enter your email
type_field = driver.find_element('id', 'email-login')
time.sleep(1.1) 
type_field.send_keys('dominic.mortimer99@gmail.com')

time.sleep(1.1)

# click into password section
password_field = driver.find_element('xpath', '/html/body/div[1]/main/div[2]/div[1]/div[2]/div/div/form/div[2]/div[1]/label')
time.sleep(1.1) 
password_field.click()

time.sleep(1.1)

# enter password
type_field = driver.find_element('id', 'password-login')
time.sleep(1.1)
type_field.send_keys(password)

time.sleep(1.1)

# click the sign in option
sign_in = driver.find_element('xpath', '/html/body/div[1]/main/div[2]/div[1]/div[2]/div/div/form/button/span[1]/span')
time.sleep(1.1)
sign_in.click()

# allow time for page to load
time.sleep(5.1)

# click into transactions page
element = driver.find_element('xpath', "/html/body/div[1]/div/div[3]/nav/div[2]/nav/div[1]/div[1]/div[2]/div/div/div[1]/div/div/p")
driver.execute_script("arguments[0].click();", element)

In between running the above and below blocks, make sure you scroll to whatever depth you need to get the charges you want.

In [5]:
# select element that holds the transactions and extract the text
element = driver.find_element('xpath', '/html/body/div[1]/div/div[3]/main/div/div[3]/main/div')
time.sleep(1.1)
charges = element.text

In [6]:
# close and quit the driver
driver.close()
time.sleep(1.1)
driver.quit()

### Process the Raw Text Data and Extract Charges by Month

In [7]:
##################################################################################################################
# separate out the charges by month
##################################################################################################################

# initialize months of year
months_of_year = 'January|February|March|April|May|June|July|August|September|October|November|December'

# in list of charges, find indices for start of each month
indices_of_month_starts = [m.start() for m in re.finditer(months_of_year, charges)]

# initialize dict to hold charges by month
charges_dict = {}

# loop through all indices we pulled out up top
for i, month_charges in enumerate(indices_of_month_starts):

    ending = len(charges) if (i + 1) == len(indices_of_month_starts) else indices_of_month_starts[i + 1]
    charges_dict[charges[month_charges:month_charges + 3]] = charges[month_charges:ending]

At this point, we have a dict where the keys represent the month of interest, and the values represent the charges (as a string).  Let's loop through the months of interest and extract the charges into a df.

In [26]:
import numpy as np

In [27]:
for key in charges_dict.keys():
    
    print(key)
    
    # extract the charges for month of interest and convert to list
    charge_list = charges_dict[key].split('\n')[1:]
    charge_list = charge_list if charge_list[-1] != '' else charge_list[:-1]

    # pull out the rows which correspond to rewards earned
    reward_indices = []
    rewards = []
    for i, charge in enumerate(charge_list):
        if '+$' in charge:
            reward_indices.append(i)
            rewards.append(charge)
            
    # delete all rows which correspond to rewards earned
    for i in sorted(reward_indices, reverse=True):
        del charge_list[i]
            
  # initialize place, charge and date lists (will later be converted into df)
    places = []
    charges = []
    dates = []

    # loop through the charge list
    for i, item in enumerate(charge_list):

        # append item to places, charge or date list depending on what it is
        j = i + 1
        if j % 3 == 1:
            places.append(item)
        elif j % 3 == 2:
            charges.append(item)
        else:
            dates.append(item)
            
    # create df of charges for month of interest
    df = pd.DataFrame({
        'month' : [key] * len(dates),
        'date' : dates,
        'place' : places,
        'charge' : charges
    })
    
    # we get funny characters on lines where charges were declined, for now we'll just remove these lines
    df = df.loc[~((df['date'].str.contains(chars_to_remove)) & (df['date'].str.contains('Declined'))), :]
    
    df['date'] = np.where((df['date'].str.contains(chars_to_remove)) & (df['date'].str.contains('Pending')),
                          df['date'].str.split(chars_to_remove).str[0],
                          df['date'])
            
    break

Jan


In [28]:
df.head()

,month,date,place,charge
0,Jan,Jan 25,Esso / Circle K Canada,-$60.63
1,Jan,Jan 22,Real Canadian Superstore ON,-$46.38
2,Jan,Jan 22,Uber Online,-$45.49
3,Jan,Jan 22,Tim Hortons #8434 Scarborough Can,-$13.90
4,Jan,Jan 21,Horseshoe Tavern Queen St W,-$14.00


In [29]:
##################################################################################################################
# loop through months, extract charges into a df
##################################################################################################################

# initialize empty df
final_df = pd.DataFrame(columns=['month', 'date', 'place', 'charge'])

# loop through the months in the charges
for key in charges_dict.keys():
    
    # extract the charges for month of interest and convert to list
    charge_list = charges_dict[key].split('\n')[1:]
    charge_list = charge_list if charge_list[-1] != '' else charge_list[:-1]

    # pull out the rows which correspond to rewards earned
    reward_indices = []
    rewards = []
    for i, charge in enumerate(charge_list):
        if '+$' in charge:
            reward_indices.append(i)
            rewards.append(charge)

    # delete all rows which correspond to rewards earned
    for i in sorted(reward_indices, reverse=True):
        del charge_list[i]

    # initialize place, charge and date lists (will later be converted into df)
    places = []
    charges = []
    dates = []

    # loop through the charge list
    for i, item in enumerate(charge_list):

        # append item to places, charge or date list depending on what it is
        j = i + 1
        if j % 3 == 1:
            places.append(item)
        elif j % 3 == 2:
            charges.append(item)
        else:
            dates.append(item)
            
    # create df of charges for month of interest
    df = pd.DataFrame({
        'month' : [key] * len(dates),
        'date' : dates,
        'place' : places,
        'charge' : charges
    })
    
    # we get funny characters on lines where charges were declined, for now we'll just remove these lines
    df = df.loc[~((df['date'].str.contains(chars_to_remove)) & (df['date'].str.contains('Declined'))), :]
    
    # pending charges have a funny format, needs fixing
    df['date'] = np.where((df['date'].str.contains(chars_to_remove)) & (df['date'].str.contains('Pending')),
                      df['date'].str.split(chars_to_remove).str[0],
                      df['date'])

    # process the charge data
    df['charge'] = df['charge'].str.replace(',', '')
    df['charge'] = df['charge'].str.replace('$', '').astype('float')
    
    # process the date data
    df['date'] = month_to_year_dict[key] + ' ' + df['date']
    df['date'] = pd.to_datetime(df['date'], format='%Y %b %d')
    
    # append back to final df
    final_df = pd.concat([df, final_df])

In [30]:
final_df

,month,date,place,charge
0,Dec,2022-12-31,Hemisphere Cannabis Co Toronto Can,-20.55
1,Dec,2022-12-31,Duke's St Lawrence Market Front St E,-30.21
2,Dec,2022-12-30,Cineplex 1303 Yonge St,-126.63
3,Dec,2022-12-30,Air Canada Online,-23.63
4,Dec,2022-12-30,Air Canada Online,-19.69
5,Dec,2022-12-30,Air Canada Online,-19.69
6,Dec,2022-12-30,Air Canada Online,-23.63
7,Dec,2022-12-30,Air Canada Online,-501.34
8,Dec,2022-12-30,Air Canada Online,-501.34
9,Dec,2022-12-29,The Old Spaghetti Fact Toronto Can,-96.16


In [80]:
################################################################################
# attempt at scrolling the page using selenium, wasn't working at last check
################################################################################

# SCROLL_PAUSE_TIME = 0.5

# # Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # Wait to load page
#     time.sleep(SCROLL_PAUSE_TIME)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height